In [1]:
NODE_COUNT = 8
LOG_DIR = f"/home/<User>/repos/csg_is/calculating_scores/DFL/RING/fedstellar_DFL_18_07_2024_14_42_44/metrics/participant_"
date_str = "18/07/2024 12:42:44	" # minus two hours for timezone

In [2]:
import os
import pandas as pd
from datetime import datetime

# import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def extract_scalars_from_event_file(event_file):
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()
    

    tags = event_acc.Tags()["scalars"]

    all_scalars = {}
    for tag in tags:
        events = event_acc.Scalars(tag)
        values = [event.value for event in events]
        steps = [event.step for event in events]

        timestamps = [pd.to_datetime(event.wall_time, unit='s') for event in events]
        all_scalars[tag] = pd.DataFrame({"step": steps, "value": values, "timestamp": timestamps})

    return all_scalars


files_t = []
def event_files_to_dataframe(directory):
    all_scalars_dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.startswith("events.out.tfevents"):
                event_file = os.path.join(root, file)

                file_from = pd.to_datetime(file.split(".")[3], unit='s')
                files_t.append(file_from)

                # print(file_from)
                if file_from < date_obj:
                    continue

                scalars = extract_scalars_from_event_file(event_file)
                for tag, df in scalars.items():
                    df["tag"] = tag
                    all_scalars_dfs.append(df)

    all_scalars_df = pd.concat(all_scalars_dfs, ignore_index=True)
    return all_scalars_df

df_all = []
for i in range(NODE_COUNT):
    # Directory containing your event files
    log_dir = LOG_DIR +f"{i}"
    print(log_dir)

    date_obj = datetime.strptime(date_str.rstrip(" ").rstrip("	"), "%d/%m/%Y %H:%M:%S")

    df = event_files_to_dataframe(log_dir)
    print(df.head())

    df.sort_values(by="timestamp", inplace=True)

    files_t.sort(reverse=True)
    print(files_t[0], files_t[-1])

    df[df["tag"]=='Test/Loss']["timestamp"].max()

    test_df = df[df["tag"].str.startswith("Test/")].groupby("tag").last().copy()
    print(test_df)
    test_df["participant"] = f"participant_{i}"
    df_all.append(test_df)

/home/<User>/repos/csg_is/calculating_scores/DFL/RING/fedstellar_DFL_18_07_2024_14_42_44/metrics/participant_0
   step     value                     timestamp         tag
0  6067  0.283488 2024-07-18 12:54:12.616409856  Train/Loss
1  6117  0.126709 2024-07-18 12:54:13.515448064  Train/Loss
2  6167  0.201633 2024-07-18 12:54:14.421704448  Train/Loss
3  6217  0.170995 2024-07-18 12:54:15.361655040  Train/Loss
4  6267  0.096924 2024-07-18 12:54:16.134575104  Train/Loss
2024-07-18 12:56:58 2024-07-18 12:43:03
                step     value                     timestamp
tag                                                         
Test/Accuracy   7616  0.789455 2024-07-18 12:56:49.909832448
Test/F1Score    7616  0.756295 2024-07-18 12:56:49.909864704
Test/Loss       7616  0.811499 2024-07-18 12:56:49.909801216
Test/Precision  7616  0.775220 2024-07-18 12:56:49.909843712
Test/Recall     7616  0.789455 2024-07-18 12:56:49.909854976
/home/<User>/repos/csg_is/calculating_scores/DFL/RING/fedstell

In [3]:
concat_df = pd.concat(df_all)
concat_df

,step,value,timestamp,participant
tag,,,,
Test/Accuracy,7616,0.789455,2024-07-18 12:56:49.909832448,participant_0
Test/F1Score,7616,0.756295,2024-07-18 12:56:49.909864704,participant_0
Test/Loss,7616,0.811499,2024-07-18 12:56:49.909801216,participant_0
Test/Precision,7616,0.775220,2024-07-18 12:56:49.909843712,participant_0
Test/Recall,7616,0.789455,2024-07-18 12:56:49.909854976,participant_0
Test/Accuracy,4873,0.875504,2024-07-18 12:56:51.325873664,participant_1
Test/F1Score,4873,0.839712,2024-07-18 12:56:51.325904640,participant_1
Test/Loss,4873,1.188256,2024-07-18 12:56:51.325832960,participant_1
Test/Precision,4873,0.844686,2024-07-18 12:56:51.325884672,participant_1


In [4]:
concat_df.groupby("tag").mean().round(4)

/tmp/ipykernel_42455/4276830878.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,6213.25,0.8204
Test/F1Score,6213.25,0.7901
Test/Loss,6213.25,0.9718
Test/Precision,6213.25,0.8138
Test/Recall,6213.25,0.8204


In [5]:
concat_df.groupby("tag").std().round(2)

/tmp/ipykernel_42455/582865284.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").std().round(2)


,step,value
tag,,
Test/Accuracy,1441.82,0.04
Test/F1Score,1441.82,0.04
Test/Loss,1441.82,0.22
Test/Precision,1441.82,0.04
Test/Recall,1441.82,0.04


In [6]:
test_results = concat_df.groupby("tag").mean().round(4)
test_results[test_results.index.str.startswith("Test/")]

/tmp/ipykernel_42455/1227319542.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_results = concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,6213.25,0.8204
Test/F1Score,6213.25,0.7901
Test/Loss,6213.25,0.9718
Test/Precision,6213.25,0.8138
Test/Recall,6213.25,0.8204
